In [ ]:
import pandas as pd
import os
import numpy as np
import joblib
import random
import time
from joblib import Parallel, delayed
from tqdm import tqdm, trange



df = pd.read_excel(r"D:\Proyecto_proyección_estados\3. Panel\panel_estudiantes_v12-15-10-2025.xlsx")

# # Cargar modelo y label encoder
clf = joblib.load(r"D:\Proyecto_proyección_estados\4. Modelo de clasificación\mejor_modelo_randomforest.pkl")
le = joblib.load(r"D:\Proyecto_proyección_estados\4. Modelo de clasificación\mejor_labelencoder.pkl")

In [4]:
# ==========================
# Estados absorbentes
# ==========================
ABSORBENTES = {"Graduado", "Abandono", "Pérdida_del_beneficio", "Sin_bolsa_de_creditos"}

# ==========================
# Función de predicción (reglas + modelo)
# ==========================
def predecir_estado(nuevo, clf, le):
    resultados = []
    for _, row in nuevo.iterrows():
        # ====== REGLAS ======
        if row["N_Aplazado"] >= 4:
            resultados.append("Abandono")
        elif row["N_Abandono"] >= 1:
            resultados.append("Abandono")
        elif row["N_Sin_bolsa_de_creditos"] >= 1:
            resultados.append("Sin_bolsa_de_creditos")
        elif (row["pct_aprob_acum"] + row["pct_perd_acum"] >= 1.1) and \
             (row["N_Periodos_adicionales"] < 4) and \
             (row["pct_perd_acum"] > 0.1):
            resultados.append("Sin_bolsa_de_creditos")
        elif row["N_Periodos_adicionales"] > 4:
            resultados.append("Pérdida_del_beneficio")
        elif row["N_Pérdida_del_beneficio"] >= 1:
            resultados.append("Pérdida_del_beneficio")
        elif row["N_Graduado"] >= 1:
            resultados.append("Graduado")
        elif row["pct_aprob_acum"] >= 0.95:   # Graduado si llega a 95%
            resultados.append("Graduado")
        else:
            # ====== MODELO ======
            X_input = pd.DataFrame([row[clf.feature_names_in_].to_dict()])
            pred = clf.predict(X_input)
            resultados.append(le.inverse_transform(pred)[0])
    return resultados

In [ ]:
# df_original = df.copy()

In [ ]:
df = df_original[]

In [ ]:
df["pct_aprob_acum"] = df["pct_aprob_acum"].fillna(0)
df_filtrado = df[df["CONVOCATORIA"].isin(["JU1", "JU2", "JU3", "JU4", "JU5", "JU6"])]


Proyección para aprobación de  JU1 a JU6 

In [21]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit

# función logarítmica
def log_func(x, a, b):
    return a * np.log(x) + b

resultados = []

for doc, grupo in df_filtrado.groupby("DOCUMENTO"):
    grupo = grupo.sort_values("periodo_orden")
    x = grupo["periodo_orden"].values
    y = grupo["pct_aprob_acum"].values

    # Modelo lineal
    lin_model = LinearRegression().fit(x.reshape(-1, 1), y)
    x_pred = np.arange(x.min(), x.max() + 50)  # proyección extendida
    lin_pred = np.clip(lin_model.predict(x_pred.reshape(-1, 1)), 0, 100)

    # Modelo logarítmico
    try:
        popt, _ = curve_fit(log_func, x, y, maxfev=10000)
        log_pred = np.clip(log_func(x_pred, *popt), 0, 100)
    except RuntimeError:
        log_pred = np.full(len(lin_pred), np.nan)

    # Promedio de ambos
    prom_pred = np.nanmean([lin_pred, log_pred], axis=0)
    prom_pred = np.clip(prom_pred, 0, 100)

    # Cortar hasta 100%
    max_len = np.where(prom_pred >= 100)[0]
    n_periodos = max_len[0] + 1 if len(max_len) > 0 else len(prom_pred)
    x_pred = x_pred[:n_periodos]

    df_temp = pd.DataFrame({
        "DOCUMENTO": doc,
        "periodo_orden": x_pred,
        "pct_lineal": lin_pred[:n_periodos],
        "pct_log": log_pred[:n_periodos],
        "pct_promedio": prom_pred[:n_periodos]
    })
    resultados.append(df_temp)

# Combinar todo
proyecciones_aprobacion_JU1_JU6 = pd.concat(resultados, ignore_index=True)

# Guardar resultado
# proyecciones.to_parquet("proyecciones_creditos_por_estudiante.parquet", index=False)


C:\Users\Alejo\AppData\Local\Temp\ipykernel_23544\1390437914.py:24: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(log_func, x, y, maxfev=10000)
C:\Users\Alejo\AppData\Local\Temp\ipykernel_23544\1390437914.py:24: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(log_func, x, y, maxfev=10000)
C:\Users\Alejo\AppData\Local\Temp\ipykernel_23544\1390437914.py:24: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(log_func, x, y, maxfev=10000)
C:\Users\Alejo\AppData\Local\Temp\ipykernel_23544\1390437914.py:24: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(log_func, x, y, maxfev=10000)
C:\Users\Alejo\AppData\Local\Temp\ipykernel_23544\1390437914.py:24: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(log_func, x, y, maxfev=10000)
C:\Users\Alejo\AppData\Local\Temp\ipykernel_23544\13904

Proyección para pérdida de  JU1 a JU6 

In [ ]:

proyecciones_aprobacion_JU1_JU6

In [14]:
# función logarítmica
def log_func(x, a, b):
    return a * np.log(x) + b

resultados = []

for doc, grupo in df_filtrado.groupby("DOCUMENTO"):
    grupo = grupo.sort_values("periodo_orden")
    x = grupo["periodo_orden"].values
    y = grupo["pct_perd_acum"].values

    # Modelo lineal
    lin_model = LinearRegression().fit(x.reshape(-1, 1), y)
    x_pred = np.arange(x.min(), x.max() + 50)  # proyección extendida
    lin_pred = np.clip(lin_model.predict(x_pred.reshape(-1, 1)), 0, 100)

    # Modelo logarítmico
    try:
        popt, _ = curve_fit(log_func, x, y, maxfev=10000)
        log_pred = np.clip(log_func(x_pred, *popt), 0, 100)
    except RuntimeError:
        log_pred = np.full(len(lin_pred), np.nan)

    # Promedio de ambos
    prom_pred = np.nanmean([lin_pred, log_pred], axis=0)
    prom_pred = np.clip(prom_pred, 0, 100)

    # Cortar hasta 100%
    max_len = np.where(prom_pred >= 100)[0]
    n_periodos = max_len[0] + 1 if len(max_len) > 0 else len(prom_pred)
    x_pred = x_pred[:n_periodos]

    df_temp = pd.DataFrame({
        "DOCUMENTO": doc,
        "periodo_orden": x_pred,
        "pct_lineal": lin_pred[:n_periodos],
        "pct_log": log_pred[:n_periodos],
        "pct_promedio": prom_pred[:n_periodos]
    })
    resultados.append(df_temp)

# Combinar todo
proyecciones_perdida_JU1_JU6 = pd.concat(resultados, ignore_index=True)

C:\Users\Alejo\AppData\Local\Temp\ipykernel_23544\1162631030.py:19: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(log_func, x, y, maxfev=10000)
C:\Users\Alejo\AppData\Local\Temp\ipykernel_23544\1162631030.py:19: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(log_func, x, y, maxfev=10000)
C:\Users\Alejo\AppData\Local\Temp\ipykernel_23544\1162631030.py:19: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(log_func, x, y, maxfev=10000)
C:\Users\Alejo\AppData\Local\Temp\ipykernel_23544\1162631030.py:19: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(log_func, x, y, maxfev=10000)
C:\Users\Alejo\AppData\Local\Temp\ipykernel_23544\1162631030.py:19: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(log_func, x, y, maxfev=10000)
C:\Users\Alejo\AppData\Local\Temp\ipykernel_23544\11626

In [22]:
proyecciones_aprobacion_JU1_JU6
proyecciones_perdida_JU1_JU6

,DOCUMENTO,periodo_orden,pct_lineal,pct_log,pct_promedio
0,478541,1,0.000000,0.000000,0.000000
1,478541,2,0.000000,0.000000,0.000000
2,478541,3,0.000000,0.000000,0.000000
3,478541,4,0.000000,0.000000,0.000000
4,478541,5,0.000000,0.000000,0.000000
...,...,...,...,...,...
1987358,10288661011,51,0.907429,0.321287,0.614358
1987359,10288661011,52,0.922743,0.322254,0.622499
1987360,10288661011,53,0.938057,0.323203,0.630630
1987361,10288661011,54,0.953371,0.324135,0.638753


In [43]:
df_panel = df_filtrado[df_filtrado["DOCUMENTO"].isin([881711, 5084270, 1000002496])]
proy_aprob = proyecciones_aprobacion_JU1_JU6
proy_perd =proyecciones_perdida_JU1_JU6

In [44]:
print("Columnas de proy_aprob:\n", proy_aprob.columns.tolist())
print("Columnas de proy_perd:\n", proy_perd.columns.tolist())


Columnas de proy_aprob:
 ['DOCUMENTO', 'periodo_orden', 'pct_lineal', 'pct_log', 'pct_promedio']
Columnas de proy_perd:
 ['DOCUMENTO', 'periodo_orden', 'pct_lineal', 'pct_log', 'pct_promedio']


In [ ]:

# ==============================
# Semilla reproducible
# ==============================
random.seed(16102025)
np.random.seed(16102025)

# ==============================
# Escenarios de proyección
# ==============================
escenario1 = "promedio"
escenario2 = "log"

col_aprob = f"pct_{escenario1}"
col_perd  = f"pct_{escenario2}"

ABSORBENTES = {"Graduado", "Abandono", "Pérdida_del_beneficio", "Sin_bolsa_de_creditos"}

# ==============================
# Probabilidades empíricas
# ==============================
prob_aplazado = {1: 0.15, 2: 0.09, 3: 0.07, 4: 0.08, 5: 0.07, 6: 0.03, 7: 0.03}
prob_abandono = {1: 0.02, 2: 0.09, 3: 0.16, 4: 0.20, 5: 0.25, 6: 0.27, 7: 0.30}

# ==============================
# Preindexar proyecciones
# ==============================
def preindex_proyecciones(df, col):
    data = {}
    for doc, subdf in df.groupby("DOCUMENTO"):
        data[doc] = dict(zip(subdf["periodo_orden"], subdf[col]))
    return data

# ==============================
# Predicción (modelo + reglas)
# ==============================
def predecir_estado(row, clf, le):
    if row["N_Aplazado"] >= 4:
        return "Abandono"
    if row["N_Abandono"] >= 1:
        return "Abandono"
    if row["N_Sin_bolsa_de_creditos"] >= 1:
        return "Sin_bolsa_de_creditos"
    if (row["pct_aprob_acum"] + row["pct_perd_acum"] >= 1.1) and row["N_Periodos_adicionales"] < 4 and row["pct_perd_acum"] > 0.1:
        return "Sin_bolsa_de_creditos"
    if row["N_Periodos_adicionales"] > 4 or row["N_Pérdida_del_beneficio"] >= 1:
        return "Pérdida_del_beneficio"
    if row["N_Graduado"] >= 1 or row["pct_aprob_acum"] >= 0.95:
        return "Graduado"

    # Modelo
    X_input = pd.DataFrame([{k: row[k] for k in clf.feature_names_in_}])
    pred = le.inverse_transform(clf.predict(X_input))[0]
    periodo = int(row["periodo_orden"])

    # Ajuste probabilístico
    if pred == "Aplazado":
        prev_aplaz = row.get("N_Aplazado", 0)
        if prev_aplaz < 1:
            p = prob_aplazado.get(min(periodo, 7), 0.03)
            if random.random() >= p:
                pred = "Matriculado"
    elif pred == "Abandono":
        p = prob_abandono.get(min(periodo, 7), 0.30)
        if random.random() >= p:
            pred = "Matriculado"

    return pred

# ==============================
# Simulación individual
# ==============================
def simular_trayectoria_proy_fast(fila, clf, le, proy_aprob_idx, proy_perd_idx, col_aprob, col_perd, max_iter=20, show_progress=False):
    fila = dict(fila)
    doc = fila["DOCUMENTO"]
    trayectoria = []

    estado = predecir_estado(fila, clf, le)
    fila["estado"] = "Matriculado"
    fila["estado_next"] = estado

    iterator = trange(max_iter, desc=f"Est. {doc}", disable=not show_progress)

    for _ in iterator:
        estado_actual = fila["estado_next"]
        if estado_actual in ABSORBENTES:
            break

        fila["estado"] = estado_actual
        fila["periodo_orden"] += 1

        # ====== Solo actualizar proyecciones si sigue matriculado ======
        if estado_actual == "Matriculado":
            periodo = fila["periodo_orden"]
            fila["pct_aprob_acum"] = proy_aprob_idx.get(doc, {}).get(periodo, fila["pct_aprob_acum"])
            fila["pct_perd_acum"]  = proy_perd_idx.get(doc, {}).get(periodo, fila["pct_perd_acum"])

        # ====== Contadores ======
        if estado_actual == "Matriculado":
            fila["N_Matriculado"] += 1
        elif estado_actual == "Aplazado":
            fila["N_Aplazado"] += 1
        elif estado_actual == "Abandono":
            fila["N_Abandono"] += 1
        elif estado_actual == "Graduado":
            fila["N_Graduado"] += 1
        elif estado_actual == "Pérdida_del_beneficio":
            fila["N_Pérdida_del_beneficio"] += 1
        elif estado_actual == "Sin_bolsa_de_creditos":
            fila["N_Sin_bolsa_de_creditos"] += 1

        # Recalcular adicionales
        fila["N_Periodos_adicionales"] = max(fila["periodo_orden"] - fila["PERIODOS_BD_SNIES"], 0)
        fila["N_Matriculas_adicionales"] = max(fila["N_Matriculado"] - fila["PERIODOS_BD_SNIES"], 0)

        # Actualizar periodo_key
        anio, sem = divmod(fila["periodo_key"], 10)
        fila["periodo_key"] = anio * 10 + (2 if sem == 1 else 1)

        # Predecir siguiente estado
        fila["estado_next"] = predecir_estado(fila, clf, le)
        trayectoria.append(fila.copy())

        # ====== Si el nuevo estado es absorbente ======
        if fila["estado_next"] in ABSORBENTES:
            fila["estado"] = fila["estado_next"]
            fila["periodo_orden"] += 1
            anio, sem = divmod(fila["periodo_key"], 10)
            fila["periodo_key"] = anio * 10 + (2 if sem == 1 else 1)
            trayectoria.append(fila.copy())
            break

    return pd.DataFrame(trayectoria)

# ==============================
# Ejecución paralela con barra global
# ==============================
def ejecutar_simulacion(df_filtrado, clf, le, proy_aprob_idx, proy_perd_idx, n_jobs=-1):
    ultimas_filas = df_filtrado.groupby("DOCUMENTO").tail(1).to_dict("records")

    start = time.time()
    resultados_x = Parallel(n_jobs=n_jobs, backend="loky")(
        delayed(simular_trayectoria_proy_fast)(
            fila, clf, le, proy_aprob_idx, proy_perd_idx, col_aprob, col_perd, max_iter=20, show_progress=False
        )
        for fila in tqdm(ultimas_filas, desc="Simulando estudiantes", unit="estudiante")
    )
    duracion = (time.time() - start) / 60
    print(f"\n⏱️ Simulación completada en {duracion:.2f} minutos")

    return pd.concat(resultados_x, ignore_index=True)

# ==============================
# Ejemplo de ejecución
# ==============================
# clf = joblib.load(r"D:\Proyecto_proyección_estados\4. Modelo de clasificación\mejor_modelo_randomforest.pkl")
# le = joblib.load(r"D:\Proyecto_proyección_estados\4. Modelo de clasificación\mejor_labelencoder.pkl")
# proy_aprob = pd.read_parquet(r"D:\Proyecto_proyección_estados\proyecciones_aprobacion_JU1_JU6.parquet")
# proy_perd = pd.read_parquet(r"D:\Proyecto_proyección_estados\proyecciones_perdida_JU1_JU6.parquet")
# df_filtrado = pd.read_parquet(r"D:\Proyecto_proyección_estados\3. Panel\panel_estudiantes_filtrado.parquet")

proy_aprob_idx = preindex_proyecciones(proy_aprob, col_aprob)
proy_perd_idx  = preindex_proyecciones(proy_perd,  col_perd)

resultados_df = ejecutar_simulacion(df_filtrado, clf, le, proy_aprob_idx, proy_perd_idx, n_jobs=-1)
# resultados_df.to_parquet(r"D:\Proyecto_proyección_estados\5. Resultados\trayectorias_optimizadas.parquet", index=False)



⏱️ Simulación completada en 282.26 minutos


In [ ]:
resultados_df.to_excel(r"D:\Proyecto_proyección_estados\5. Proyección\JJU1-JU6-proyeccion.xlsx")